In [ ]:
# Authenticate API
import requests
import pprint
import json
import time

auth_key = 'a6cc25c4b313444ebb1c12c1ad27d354'

upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'

headers_auth_only = {'authorization': auth_key}

headers = {
    "authorization": auth_key,
    "content-type": "application/json"
}

CHUNK_SIZE = 5_242_880  # 5MB


In [ ]:
def upload(filename):
    def read_file(filename):
        with open(filename, 'rb') as f:
            while True:
                data = f.read(CHUNK_SIZE)
                if not data:
                    break
                yield data
    upload_response = requests.post(upload_endpoint, headers=headers_auth_only, data=read_file(filename))
    pprint.pprint(upload_response.json())
    return upload_response.json()['upload_url']


In [ ]:
def transcribe(audio_url):
    transcript_request = {
        'audio_url': audio_url,
        'auto_chapters': True,
        'auto_highlights': True
    }

    transcript_response = requests.post(transcript_endpoint,
                                        json=transcript_request,
                                        headers=headers)
    pprint.pprint(transcript_response.json())
    return transcript_response.json()['id']

In [ ]:
def poll(transcript_id):
    polling_endpoint = transcript_endpoint + '/' + transcript_id
    polling_response = requests.get(polling_endpoint, headers=headers)

    if polling_response.json()['status'] == 'completed':
        filename = transcript_id + '.txt'
        with open(filename, 'w') as f:
            f.write(polling_response.json()['text'])

        filename = transcript_id + '_chapters.json'
        with open(filename, 'w') as f:
            chapters = polling_response.json()['chapters']
            json.dump(chapters, f, indent=4)
            
        filename = transcript_id + '_highlights.json'
        with open(filename, 'w') as f:
            data = polling_response.json()['auto_highlights_result']
            json.dump(data, f, indent=4)

        print('Transcript saved')
        return True
    return False

In [ ]:
def pipeline(filename):
    audio_url = upload(filename)
    transcribe_id = transcribe(audio_url)
    while True:
        result = poll(transcribe_id)
        if result:
            break
        print("waiting for 60 seconds")
        time.sleep(60)


if __name__ == '__main__':
   pipeline('/content/drive/MyDrive/DSC_audio.mp3')

{'upload_url': 'https://cdn.assemblyai.com/upload/d43b5494-ceef-4976-b514-44480ecc3f20'}
{'acoustic_model': 'assemblyai_default',
 'audio_duration': None,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/d43b5494-ceef-4976-b514-44480ecc3f20',
 'auto_chapters': True,
 'auto_highlights': True,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'cluster_id': None,
 'confidence': None,
 'content_safety': False,
 'content_safety_labels': {},
 'custom_spelling': None,
 'disfluencies': False,
 'dual_channel': None,
 'entities': None,
 'entity_detection': False,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {},
 'id': 'rbl19gtgt7-18ea-46ed-a4ec-b16f364df22b',
 'language_code': 'en_us',
 'language_detection': False,
 'language_model': 'assemblyai_default',
 'punctuate': True,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redac